<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 03 - Creating and Querying a Data Mart

In this tutorial you will learn how to create and query a Data Mart using Virtualized Data in a SQL Server big data cluster. 

Wide World Importers is interested in ingesting the data from web logs from an HDFS source where they have been streamed. They want to be able to analyze the traffic to see if there is a pattern in time, products or locations. 

The web logs, however, are refreshed periodically. WWI would like to keep the logs in local storage to do deeper analysis. 

In this Jupyter Notebook you'll create a location to store the log files as a SQL Server Table in the SQL Data Pool, and then fill it by creating an External Table that reads HDFS.

In [1]:


  IF NOT EXISTS(SELECT * FROM sys.external_data_sources WHERE name = 'SqlDataPool')
      CREATE EXTERNAL DATA SOURCE SqlDataPool
      WITH (LOCATION = 'sqldatapool://controller-svc:8080/datapools/default');

: Execution failed due to an unexpected error: 
	This editor is not connected to a database
Parameter name: OwnerUri

Total execution time: 00:00:00

In [4]:
CREATE EXTERNAL TABLE [web_clickstream_clicks_data_pool]
    ("wcs_click_date_sk" BIGINT 
    , "wcs_click_time_sk" BIGINT 
    , "wcs_sales_sk" BIGINT 
    , "wcs_item_sk" BIGINT
    , "wcs_web_page_sk" BIGINT 
    , "wcs_user_sk" BIGINT)
    WITH
    (
    DATA_SOURCE = SqlDataPool,
    DISTRIBUTION = ROUND_ROBIN
    );
GO

: Msg 2714, Level 16, State 6, Line 1
There is already an object named 'web_clickstream_clicks_data_pool' in the database.

Total execution time: 00:00:00.033

In [5]:
CREATE EXTERNAL FILE FORMAT csv_file
WITH (FORMAT_TYPE = DELIMITEDTEXT,
      FORMAT_OPTIONS(
          FIELD_TERMINATOR = ',',
          STRING_DELIMITER = '"',
          FIRST_ROW = 2, 
          USE_TYPE_DEFAULT = True)
)
/* Create an External Table that can read from the Storage Pool File Location */
IF NOT EXISTS(SELECT * FROM sys.external_tables WHERE name = 'web_clickstreams_hdfs')
BEGIN
    CREATE EXTERNAL TABLE [web_clickstreams_hdfs]
    ("wcs_click_date_sk" BIGINT 
    , "wcs_click_time_sk" BIGINT 
    , "wcs_sales_sk" BIGINT 
    , "wcs_item_sk" BIGINT
    , "wcs_web_page_sk" BIGINT 
    , "wcs_user_sk" BIGINT)
    WITH
    (
        DATA_SOURCE = SqlStoragePool,
        LOCATION = '/web_logs',
        FILE_FORMAT = csv_file
    );
END

Commands completed successfully.

Total execution time: 00:00:00.341

In [6]:
BEGIN
   INSERT INTO web_clickstream_clicks_data_pool
   SELECT wcs_click_date_sk
    , wcs_click_time_sk 
    , wcs_sales_sk 
    , wcs_item_sk 
    , wcs_web_page_sk  
    , wcs_user_sk 
     FROM web_clickstreams_hdfs
END

(6770549 rows affected)

Total execution time: 00:00:45.350

In [7]:
SELECT count(*) AS TotalRecords FROM [dbo].[web_clickstream_clicks_data_pool]
SELECT TOP 10 * FROM [dbo].[web_clickstream_clicks_data_pool]

(1 row affected)

(10 rows affected)

Total execution time: 00:00:03.568

TotalRecords
6770549


wcs_click_date_sk,wcs_click_time_sk,wcs_sales_sk,wcs_item_sk,wcs_web_page_sk,wcs_user_sk
37757,46857,NULL,7679,7,NULL
37757,81346,NULL,8318,7,NULL
37757,36787,NULL,17245,7,NULL
37757,54811,NULL,12518,7,NULL
37758,72062,NULL,10360,7,NULL
37758,52272,NULL,13616,7,NULL
37758,49261,NULL,12336,7,NULL
37758,75152,NULL,3398,7,NULL
37758,6933,NULL,14750,7,NULL
37758,22472,NULL,9197,7,NULL


## Next Steps: Continue on to Working with Spark and ETL

Now you're ready to open the next Python Notebook - `notebook_04.ipynb` - to learn how to create and work with Spark and Extracting, Transforming and Loading data.